## Declare Models

Here, we define _module-level_ constructs that will __form the structures__ which we will be `querying` from the database. This structure, known as a `Declarative Mapping`, _defines at once_ both a __Python object model__, as well as __database metadata__ that _describes real SQL tables_ that `exist, or will exist`, in a particular database.

In [1]:
from sqlalchemy import Column, ForeignKey, Integer, String, create_engine, select
from sqlalchemy.orm import Session, declarative_base, relationship

In [2]:
Base = declarative_base()

In [3]:
class User(Base):
    __tablename__ = "user_account"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address", back_populates="user", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [4]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"), nullable=False)
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

The _mapping_ starts with a `base class`, which above is called `Base`, and is created by calling upon the `declarative_base()` function, which produces a __new `base` class__.

_Individual mapped classes_ are then _created_ by making __subclasses of `Base`__. A _mapped class_ typically refers to a `single particular database table`, the name of which is indicated by using the __`__tablename__`__ _class-level attribute_.

Next, _columns_ that are part of the table are declared, by _adding attributes_ __linked__ to the `Column` construct. `Column` describes _all aspects_ of a `database column`, including __typing information__ with _type objects_ such as `Integer` and `String` as well as __server defaults__ and __constraint information__, such as _membership_ within the `primary key` and `foreign keys`.

_All_ `ORM mapped classes` __require__ _at least_ __one column__ be declared as part of the `primary key`, typically by using the `Column.primary_key` parameter on those `Column` objects that should be part of the key. In the above example, the `User.id` and `Address.id` columns are _marked_ as __primary key__.

Taken together, the _combination_ of a `string table name` as well as a `list of column declarations` is __referred__ towards in SQLAlchemy as __`table metadata`__. Setting up `table metadata` using both `Core` and `ORM` approaches is introduced in the `SQLAlchemy 1.4/2.0 Tutorial` at `Working with Database Metadata`. The above mapping is an example of what's referred towards as `Declarative Table configuration`.

Other _Declarative directives_ are available, most commonly the `relationship()` construct indicated above. In contrast to the _column-based attributes_, `relationship()` denotes a __linkage between two `ORM classes`__. In the above example, `User.addresses` __links `User` to `Address`__, and `Address.user` __links `Address` to `User`__. The `relationship()` construct is introduced in the `SQLAlchemy 1.4/2.0 Tutorial` at `Working with Related Objects`.

Finally, the above example classes include a `__repr__()` method, which is _not required_ but is __useful for debugging__.

## Create an Engine

The `Engine` is a _factory_ that can __create new database connections__ for us, which also _holds onto connections_ inside of a `Connection Pool` for __fast reuse__. For learning purposes, we normally use a `SQLite` __memory-only__ database for convenience.

In [5]:
engine = create_engine("sqlite://", echo=True, future=True)

> ##### Tips
> 
> The `echo=True` parameter indicates that __SQL emitted by connections__ will be _logged to standard out_. `future=True` is to __ensure__ we are using the _latest SQLAlchemy 2.0-style APIs_.

## Emit CREATE TABLE DDL

Using our `table metadata` and our `engine`, we can _generate_ our `schema` __at once__ in our __target `SQLite` database__, using a method called `MetaData.create_all()`.

In [6]:
Base.metadata.create_all(engine)

2022-10-14 11:00:49,643 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-14 11:00:49,645 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-10-14 11:00:49,646 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-14 11:00:49,647 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-10-14 11:00:49,648 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-14 11:00:49,649 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-10-14 11:00:49,650 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-14 11:00:49,651 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-10-14 11:00:49,653 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-14 11:00:49,655 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-10-14 11:00:49,656 INFO sqlalchemy.engine.Engine [no key 0.00064s] ()
2022-10-14 11:00:49,657 INFO sqlalchemy.engine.Engine 
C

## Create Objects and Persist

We are now ready to `insert` data in the database. We accomplish this by _creating instances_ of `User` and `Address` classes, which have an `__init__()` method already as __established automatically__ by the __`declarative mapping process`__. We then _pass them to the database_ using an object called a `Session`, which makes use of the `Engine` to __interact with the `database`__. The `Session.add_all()` method is used here to __add multiple objects at once__, and the `Session.commit()` method will be used to __flush any pending changes__ to the database and then __commit__ the _current database transaction_, which is __always in progress__ whenever the `Session` is used.

In [7]:
with Session(engine) as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")]
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ]
    )
    patrick = User(name="patrick", fullname="Patrick Star")
    
    session.add_all([spongebob, sandy, patrick])
    session.commit()

2022-10-14 11:00:49,722 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-14 11:00:49,724 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-10-14 11:00:49,725 INFO sqlalchemy.engine.Engine [generated in 0.00111s] ('spongebob', 'Spongebob Squarepants')
2022-10-14 11:00:49,726 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-10-14 11:00:49,728 INFO sqlalchemy.engine.Engine [cached since 0.003833s ago] ('sandy', 'Sandy Cheeks')
2022-10-14 11:00:49,730 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-10-14 11:00:49,732 INFO sqlalchemy.engine.Engine [cached since 0.008028s ago] ('patrick', 'Patrick Star')
2022-10-14 11:00:49,735 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2022-10-14 11:00:49,737 INFO sqlalchemy.engine.Engine [generated in 0.00180s] ('spongebob@sqlalchemy.org', 1)
2022-10-14 11:00:49,740 INFO sqlalchemy

> ##### Tips
> 
> It's recommended that the `Session` be __used in context manager style__ as above, that is, using the Python `with: statement`. The `Session` object __represents__ _active database resources_ so it's good to make sure it's __closed out__ when a _series of operations_ are __`completed`__. In the next section, we'll keep a `Session` opened just for illustration purposes.

## Simple SELECT

With some rows in the database, here's the _simplest form_ of __emitting__ a `SELECT` statement _to load some objects_. To create `SELECT` statements, we use the `select()` function to __create a new `Select` object__, which we then _invoke_ using a `Session`. The method that is _often useful_ when __querying for `ORM` objects__ is the `Session.scalars()` method, which will return a `ScalarResult` object that will _iterate_ through the ORM objects we've selected.

In [8]:
session = Session(engine)

stmt = select(User).where(User.name.in_(["spongebob", "sandy"]))
for user in session.scalars(stmt):
    print(user)

2022-10-14 11:00:49,839 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-14 11:00:49,842 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name IN (?, ?)
2022-10-14 11:00:49,844 INFO sqlalchemy.engine.Engine [generated in 0.00194s] ('spongebob', 'sandy')
User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')


The above query also made use of the `Select.where()` method to __add `WHERE` criteria__, and also used the `ColumnOperators.in_()` method that's part of all SQLAlchemy _column-like constructs_ to use the SQL `IN operator`.

## SELECT with JOIN

It's _very common_ to `query` amongst __multiple tables at once__, and in SQL the `JOIN` _keyword_ is the _primary way_ this happens. The `Select` construct __creates joins__ using the `Select.join()` method.

In [9]:
stmt = (
    select(Address).
    join(Address.user).
    where(User.name == "sandy").
    where(Address.email_address == "sandy@sqlalchemy.org")
)
sandy_address = session.scalars(stmt).one()
print(f"{sandy_address = }")

2022-10-14 11:00:49,921 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id 
FROM address JOIN user_account ON user_account.id = address.user_id 
WHERE user_account.name = ? AND address.email_address = ?
2022-10-14 11:00:49,922 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ('sandy', 'sandy@sqlalchemy.org')
sandy_address = Address(id=2, email_address='sandy@sqlalchemy.org')


The above query illustrates __multiple `WHERE` criteria__ which are __automatically chained together__ using `AND`, as well as how to use SQLAlchemy _column-like objects_ to __create `"equality"` comparisons__, which uses the _overridden_ Python method `ColumnOperators.__eq__()` to produce a _SQL criteria object_.

## Make Changes

The `Session` object, in conjunction with our _ORM-mapped classes_ `User` and `Address`, __automatically track changes to the objects__ as they are made, which result in _SQL statements_ that will be __emitted the next time the `Session` flushes__. Below, we _change_ one `email address` _associated_ with `"sandy"`, and also __add a new email address__ to `"patrick"`, _after emitting_ a `SELECT` to __retrieve the row for `"patrick"`__.

In [10]:
stmt = select(User).where(User.name == "patrick")
patrick = session.scalars(stmt).one()
patrick.addresses.append(Address(email_address="patrickstar@sqlalchemy.org"))
sandy_address.email_address = "sandy_cheeks@sqlalchemy.org"
session.commit()

2022-10-14 11:00:50,002 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-10-14 11:00:50,003 INFO sqlalchemy.engine.Engine [generated in 0.00146s] ('patrick',)
2022-10-14 11:00:50,008 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-10-14 11:00:50,009 INFO sqlalchemy.engine.Engine [generated in 0.00107s] (3,)
2022-10-14 11:00:50,014 INFO sqlalchemy.engine.Engine UPDATE address SET email_address=? WHERE address.id = ?
2022-10-14 11:00:50,015 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ('sandy_cheeks@sqlalchemy.org', 2)
2022-10-14 11:00:50,017 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2022-10-14 11:00:50,018 INFO sqlalchemy.engine.Engine [cached since 0.2832s ago] ('patrickstar@sqlalchemy.org', 

Notice when we accessed `patrick.addresses`, a `SELECT` was _emitted_. This is called a __`lazy load`__. Background on different ways to _access related items_ using more or less SQL is introduced at `Loader Strategies`.